In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from dcf import ecf, ytm, pv, iac, bpv, delta, CashFlowList, PayOffModel, \
    OptionPayOffModel, FixedCashFlowPayOff
from dcf.payoffs import CashFlowDetails, RateCashFlowPayOff
from dcf.tools.dc import DateType

# `float`

In [157]:
n = 1_000_000
coupon_leg = CashFlowList.from_rate_cashflows([1.,2.,3.,4.,5.], amount_list=n, origin=0., fixed_rate=0.05)
#coupon_leg.print()
redemption_leg = CashFlowList.from_fixed_cashflows([5.], amount_list=n)
# redemption_leg.print()
bond = coupon_leg + redemption_leg
bond[4.].print()

  pay date    cashflow    notional  pay rec      fixed rate    start date    end date    year fraction
----------  ----------  ----------  ---------  ------------  ------------  ----------  ---------------
       4.0    50_000.0   1_000_000  pay                0.05           3.0         4.0              1.0


### `fit()` with year fraction

In [80]:
from dcf import CashFlowList, fit, pv
from yieldcurves import YieldCurve, AlgebraCurve, DateCurve

In [81]:
today = 0.0
cashflow_list = []
schedule = [1., 2., 3., 4., 5. ]
for i, d in enumerate(schedule):
    pay_dates = [s for s in schedule if s <= d]
    # cf = CashFlowList.from_rate_cashflows(pay_dates, 1_000_000, origin=today, fixed_rate=0.002 * i)
    cf = CashFlowList.from_fixed_cashflows([d], 1)
    cashflow_list.append(cf)

In [74]:
curve = YieldCurve.from_interpolation(schedule, [0.01, 0.009, 0.012, 0.014, 0.011])
targets = [pv(c, curve.df, today) for c in cashflow_list]

In [75]:
fit(cashflow_list, 1.0, today, price_list=targets)

{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

In [96]:
yc = YieldCurve(AlgebraCurve(0.0, inplace=True))
fit(cashflow_list, yc.df, today, price_list=targets)

AlgebraCurve(_YieldCurveAdapter.df, inplace=True)
<bound method _YieldCurveAdapter.df of YieldCurve(AlgebraCurve(_YieldCurveAdapter.df, inplace=True))>


{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

In [97]:
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve)

{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

In [77]:
yc = DateCurve(YieldCurve(AlgebraCurve(0.0, inplace=True)), origin=0.0)
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve)

{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

In [78]:
yc = DateCurve(YieldCurve(AlgebraCurve(0.0, inplace=True)), origin=0.0)
grid = [yc.year_fraction(max(cf.domain)) for cf in cashflow_list]
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve, fitting_grid=grid)

{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

### `fit()` with BusinessDate

In [8]:
from businessdate import BusinessDate, BusinessSchedule
from dcf import CashFlowList, fit, pv
from yieldcurves import YieldCurve, AlgebraCurve, DateCurve
from yieldcurves.interpolation import piecewise_linear

In [58]:
today = BusinessDate(20240101)
cashflow_list = []
schedule = BusinessSchedule(today + '1y', today + '5y', step='1y')
for i, d in enumerate(schedule):
    pay_dates = [s for s in schedule if s <= d]
    #cf = CashFlowList.from_rate_cashflows(pay_dates, 1_000_000, origin=today, fixed_rate=0.002 * i)
    cf = CashFlowList.from_fixed_cashflows([d], 1)
    cashflow_list.append(cf)

In [59]:
curve = DateCurve(YieldCurve.from_interpolation(schedule, [0.01, 0.009, 0.012, 0.014, 0.011]), origin=today)
targets = [pv(c, curve.df, today) for c in cashflow_list]

In [70]:
yc = DateCurve(YieldCurve(AlgebraCurve(0.0, inplace=True)), origin=today)
fit(cashflow_list, yc.df, today, price_list=targets)

{1.002053388090349: 0.009261635865832207,
 2.001368925393566: 0.011213039013333188,
 3.0006844626967832: 0.013473990407700934,
 4.0: 0.0117910677618071,
 5.002053388090349: 0.011000000000002522}

In [71]:
yc = DateCurve(YieldCurve(AlgebraCurve(0.0, inplace=True)), origin=today)
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve)

{1.002053388090349: 0.009261635865832207,
 2.001368925393566: 0.011213039013333188,
 3.0006844626967832: 0.013473990407700934,
 4.0: 0.0117910677618071,
 5.002053388090349: 0.011000000000002522}

In [72]:
yc = DateCurve(YieldCurve(AlgebraCurve(0.0, inplace=True)), origin=today)
grid = [yc.year_fraction(max(cf.domain)) for cf in cashflow_list]
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve, fitting_grid=grid)

{1.002053388090349: 0.009261635865832207,
 2.001368925393566: 0.011213039013333188,
 3.0006844626967832: 0.013473990407700934,
 4.0: 0.0117910677618071,
 5.002053388090349: 0.011000000000002522}

### `pv()`

In [ ]:
from yieldcurves import YieldCurve
from yieldcurves.tools import plot, lin

In [ ]:
curve = YieldCurve.from_interpolation([0.0], [0.05])
plot(lin(0, 30, 0.2), curve)

In [ ]:
pv(bond, curve.df, 0.0)

### `bpv()`

In [ ]:
from yieldcurves import YieldCurve
from yieldcurves.tools import AlgebraCurve

In [ ]:
yc = YieldCurve(AlgebraCurve(curve, inplace=True))
pv(bond, yc.df, 0.0)

In [ ]:
pv1 = pv(bond, YieldCurve(AlgebraCurve(curve)).df, 0.0)
pv2 = pv(bond, YieldCurve(AlgebraCurve(curve) + 0.0001).df, 0.0)
pv2 - pv1

In [ ]:
yc = YieldCurve(AlgebraCurve(curve, inplace=True))

In [ ]:
pv(bond, yc.df, 0.0), bpv(bond, yc.df, 0.0, delta_curve=yc.curve)

### `delta()`

In [ ]:
yc = YieldCurve(AlgebraCurve(curve, inplace=True))

In [ ]:
delta(bond, yc.df, 0.0, delta_curve=yc.curve, delta_grid=[0., 1., 2., 3., 4., 5.])

### `iac()`

In [ ]:
iac(bond, 3.25)

In [ ]:
cf_list = CashFlowList.from_fixed_cashflows([0., 1., 2., 3.], [100, 100, 100, 100])
cf_list *= 10.
cf_list.print() 
#print(cf_list)
#cf_list

In [ ]:
sod = pv(cf_list, curve, valuation_date=0.0)
sod

In [ ]:
sod - ecf(cf_list, 0.0)[0.0]

In [ ]:
float(cf_list()[0.0])

In [ ]:
cl = CashFlowList.from_rate_cashflows([1,2,3,4], 100., origin=0, fixed_rate=0.5)
cl

In [ ]:
cf = RateCashFlowPayOff(pay_date=1.0, start=1.25, end=1.5, amount=100.0, fixed_rate=0.005)
f = lambda *_: 0.05
j = cf.__json__()
print(j)
RateCashFlowPayOff.from_json(j).__copy__()()

In [ ]:
d = dict.fromkeys(map(str, range(6)), 'öakrhgjfhgqjgqklejgrh')
cd = CashFlowDetails(cashflow=1., **d)
#cd.update(d)
cd

### delta()

In [ ]:
from yieldcurves import plotter, YieldCurve
from yieldcurves.tools import AlgebraCurve
from dcf.tools.pl import piecewise_linear
c = piecewise_linear([1.,2.,3.], [2.,3.,1.])
plotter[-1:5:0.1](c)

In [ ]:
from dcf import delta
yc = YieldCurve(AlgebraCurve(0.01, inplace=True))
bond_delta = delta(bond, yc.df, 0.0, delta_curve=yc.curve, delta_grid=[0., 2., 4., 6.])
bond_bpv = bpv(bond, yc.df, 0.0, delta_curve=yc.curve)
print(yc)
bond_bpv, sum(bond_delta)

In [ ]:
curve = piecewise_linear([1.,2.,3.], [.02,.03,.01])
yc = YieldCurve(AlgebraCurve(curve, inplace=True))
shift = piecewise_linear([0.0, 1.0, 2.0], [0.0, 0.01, 0.0])
plotter[0:5](curve=curve, shifted_curve=yc.curve+shift, shift=shift)
yc.curve-shift
yc

### `fit()`

In [ ]:
from dcf import fit, CashFlowList

In [ ]:
today = 0.0
schedule = [float(p) + 1 for p in range(5)]

In [ ]:
cashflow_list = []
for d in schedule:
    pay_dates = [s for s in schedule if s <= d]
    cf = CashFlowList.from_rate_cashflows(pay_dates, 1_000_000, origin=today, fixed_rate=0.002*d)
    cashflow_list.append(cf)
cashflow_list

In [ ]:
from yieldcurves import YieldCurve, AlgebraCurve
rates = [0.01, 0.009, 0.012, 0.014, 0.011]
curve = YieldCurve.from_interpolation(schedule, rates)
pv_list = [pv(c, curve.df, today) for c in cashflow_list]
pv_list

In [ ]:
from typing import Callable, Dict, Iterable, Tuple, Any


from dcf.pricer import Pricer
from yieldcurves.tools import fit

In [ ]:
class Pricer:
    def __init__(self, 
                 discount_curve: Callable, 
                 valuation_date: DateType, 
                 payoff_model: PayOffModel | None = None,
                 cashflow_list: CashFlowList | None = None):
        self.discount_curve = discount_curve
        self.valuation_date = valuation_date
        self.payoff_model = payoff_model
        self.cashflow_list = cashflow_list or CashFlowList()

    def pv(self, cashflow_list=None):
        return pv(cashflow_list or self.cashflow_list, self.discount_curve, self.valuation_date, self.payoff_model)
        
    def bpv(self, cashflow_list=None):
        return bpv(cashflow_list or self.cashflow_list, self.discount_curve, self.valuation_date, self.payoff_model)
    
    def iac(self, cashflow_list=None):
        return iac(cashflow_list or self.cashflow_list, self.valuation_date)

    def ytm(self, cashflow_list=None, present_value=0.0):
        return ytm(cashflow_list or self.cashflow_list, self.valuation_date, present_value, self.payoff_model)

In [ ]:
def fit(curve: AlgebraCurve,
        grid: Iterable[float],
        err_func_list: Iterable[Callable],
        target_list: Iterable[float] | None = None,
        bounds=(-0.1, 0.2),
        precision=1e-8
        ) -> Dict[float, float]:
    """ fit according to calibration routine to target values """
    grid = tuple(grid)
    if target_list is None:
        target_list = [0.0] * len(grid)
    addon = piecewise_linear(grid, [0.0] * len(grid))
    curve += addon
    for t, f, v in zip(grid, err_func_list, target_list):
        # set error function
        def err(current):
            addon[t] = current
            return f() - v
        # run bracketing
        _simple_bracketing(err, *bounds, precision)
    curve -= addon
    return dict(addon.items())

In [ ]:
yc = YieldCurve(AlgebraCurve(0.0, inplace=True))
grid = [max(cf.domain) for cf in cashflow_list]
callbacks = [Pricer(yc.df, 0.0, cashflow_list=cf).pv for cf in cashflow_list]
targets = pv_list

In [ ]:
rates = fit(yc.curve, grid, callbacks, targets)
YieldCurve.from_interpolation(rates.keys(), rates.values())

In [ ]:
yc2 = YieldCurve.from_interpolation(rates.keys(), rates.values())
yc2

In [ ]:
for cf in normalized_cf.values():
    print(pv(cf, yc2.df, 0.0))

In [ ]:
yc.curve -=0.1
print(pricer(normalized_cf[1.]))
yc.curve +=0.1
yc.curve +=0.2
print(pricer(normalized_cf[1.]))
yc.curve -=0.2


In [ ]:
yc.curve -= 0.1
pricer(normalized_cf[3.]), yc

In [ ]:
rates = fit(yc.curve, normalized_cf.keys(), normalized_cf, pricer)
yc2 = YieldCurve.from_interpolation(rates.keys(), rates.values())

In [ ]:
yc = YieldCurve(AlgebraCurve(0.0, inplace=True))

b = build(cashflow_list, yc.df, 0.0, price_list=pv_list)
rates = fit(yc.curve, b.keys(), b)

yc2 = YieldCurve.from_interpolation(rates.keys(), rates.values())
c = build(cashflow_list, yc2.df, 0.0, price_list=pv_list)
print(max(abs(f()) for f in b.values()))
print(max(abs(f()) for f in c.values()))
yc2

In [ ]:
yc = YieldCurve(AlgebraCurve(0.01, inplace=True))

In [ ]:
for cfl in cashflow_list:
    eps = -0.1
    v = pv(cfl, yc.df, today)
    yc.curve + eps
    print(pv(cfl, yc.df, today) - v, end=' : ')
    yc.curve - eps - eps
    print(pv(cfl, yc.df, today) - v)
    yc.curve + eps
yc

In [ ]:
for cf in cashflow_list:
    cf.print()

In [ ]:
from dcf.tools.pl import piecewise_linear
i = 0
t = schedule[i]
cfl = cashflow_list[i]
yc = YieldCurve(AlgebraCurve(0.01, inplace=True))
addon = piecewise_linear(schedule, [0.0] * len(schedule))
yc.curve += addon
v = pv(cfl, yc.df, 0.0)
addon[t] = -0.1
print(pv(cfl, yc.df, 0.0) - v)
addon[t] = 0.2
print(pv(cfl, yc.df, 0.0) - v)
yc.curve - addon

In [ ]:
data = fit(cashflow_list, yc.df, today, fitting_curve=yc.curve, fitting_grid=schedule, present_value_list=pv_list)
data

In [ ]:
yc = YieldCurve(AlgebraCurve(0.01, inplace=True))
yc.curve += piecewise_linear(schedule, data)
for cfl, v in zip(cashflow_list, pv_list):
    print(pv(cfl, yc.df, 0.0)-v)
yc.curve - piecewise_linear(schedule, data)

# BusinessDate

In [ ]:
from businessdate import BusinessDate, BusinessSchedule
today = BusinessDate(20161231)

In [ ]:
from yieldcurves import YieldCurve, plotter
curve = YieldCurve.from_interpolation([0.0, 1.0, 2.0, 3.0, 4.0], [0.01, 0.009, 0.012, 0.014, 0.011])
f = YieldCurve.from_interpolation([0.0], [0.05], spot_price=100.0)  # spot price 100 and yield of 5%
v = YieldCurve.from_interpolation([0.0], [0.1])  # flat volatility of 10%
print(curve(-1.), curve(4.1))
plotter[-1:5](**{r"$\alpha$":curve})

In [ ]:
from yieldcurves import DateCurve
crv = DateCurve(curve, origin=today)
fwd = DateCurve(f, origin=today)
vol = DateCurve(v, origin=today)
vol

In [ ]:
from dcf import OptionPayOffModel
m = OptionPayOffModel.black76(forward_curve=fwd.price, volatility_curve=vol, valuation_date=today)

In [ ]:
from dcf import ecf, pv, CashFlowList, OptionCashFlowPayOff
expiry = today + '3m'
option = OptionCashFlowPayOff(expiry, expiry, amount=-n, strike=110.)
option_list = CashFlowList.from_option_cashflows([expiry], strike_list=110., payoff_model=m)
option = option_list[0]
option

In [ ]:
# option = OptionCashFlowPayOff(expiry, expiry, amount=1, strike=110.)
m(option, today)

In [ ]:
# 0.1025...
ecf(option_list, today, m)

In [ ]:
# 0.1022...
pv(option_list, crv.df, today, m)

# `README`

In [ ]:
from dcf.plans import amortize, outstanding

In [ ]:
today = BusinessDate(20201031)
schedule = BusinessSchedule(today, today + "8q", step="1q")
start_date, payment_dates = schedule[0], schedule[1:]

number_of_payments = 8
interest_rate = 0.001
notional = 1000.

plan = amortize(number_of_payments, amount=notional)
out = outstanding(plan, amount=notional)
out

In [ ]:
principal = CashFlowList.from_fixed_cashflows([start_date], [notional])
principal

In [ ]:
redemption = CashFlowList.from_fixed_cashflows(payment_dates, plan)
redemption

In [ ]:
interest = CashFlowList.from_rate_cashflows(payment_dates, out, fixed_rate=0.001)
interest

In [ ]:
import pandas as pd
loan = interest - principal + redemption 
df = pd.DataFrame.from_records(loan(), index='pay date')
df['cashflow']

In [ ]:
df['df'] = [curve.df(t) for t in df.index]
df['pv'] = df['cashflow'] * df['df']
df

In [ ]:
df['pv'].sum()

In [ ]:
curve = YieldCurve.from_interpolation([today, today + '10y'], [-.005, .005])
curve = DateCurve(curve, origin=today)
x = BusinessSchedule(curve.origin, curve.origin +  '10y', '3m')

In [ ]:
plot(x, **{r"$\gamma(t)$":curve})

In [ ]:
pv(cashflow_list=loan, discount_curve=curve.df, valuation_date=today)  # 4.896613015654154

### `fit(option)`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from businessdate import BusinessDate
today = BusinessDate(20161231)

In [ ]:
from yieldcurves import YieldCurve, DateCurve, AlgebraCurve
curve = YieldCurve.from_interpolation([0.0, 1.0, 2.0, 3.0, 4.0], [0.01, 0.009, 0.012, 0.014, 0.011])
f = YieldCurve.from_interpolation([0.0], [0.05], spot_price=100.0)  # spot price 100 and yield of 5%
v = YieldCurve.from_interpolation([0.0], [0.1])  # flat volatility of 10%
crv = DateCurve(curve, origin=today)
fwd = DateCurve(f, origin=today)
vol = DateCurve(YieldCurve(AlgebraCurve(v, inplace=True)), origin=today)

In [ ]:
from dcf import ecf, pv, fit, CashFlowList, OptionPayOffModel
expiry = today + '3m'
m = OptionPayOffModel.black76(forward_curve=fwd.price, volatility_curve=vol, valuation_date=today)
option_list = CashFlowList.from_option_cashflows([expiry], strike_list=110., payoff_model=m)

In [ ]:
ecf(option_list, today, payoff_model=m)[expiry]

In [ ]:
pv(option_list, crv.df, today, payoff_model=m)

In [ ]:
vol.curve.curve -= 0.2
pv(option_list, crv.df, today, payoff_model=m)

In [ ]:
vol.curve.curve += 0.2
vol

In [ ]:
from dcf.tools.pl import piecewise_linear
addon = piecewise_linear([expiry], [0.0], origin=vol.origin)
addon[today + '10y'] = 0.001
addon

In [ ]:
vol

In [ ]:


data = fit([option_list], crv.df, today, fitting_curve=vol.curve.curve, fitting_grid=[expiry], present_value_list=[0.25], addon=addon)